# Importing pdfs
This notebook provides sample [EDSL](https://docs.expectedparrot.com/) code demonstrating a method `from_pdf()` that imports a PDF and automatically creates `Scenario` objects for the pages to use as parameters of survey questions. This can be helpful when using EDSL to extract qualitative information from a large text efficiently. 

EDSL is an open-source library for simulating surveys and experiments with AI agents and large language models. Please see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started.

## How it works
EDSL comes with a [variety of question types](https://docs.expectedparrot.com/en/latest/questions.html) that we can select from based on the desired form of the response (multiple choice, free text, etc.). We can also parameterize questions with textual content in order to ask questions about it. We do this by creating a `{{ placeholder }}` in a question text, e.g., *What are the key themes of this text: {{ text }}*, and then creating `Scenario` objects for the content to be inserted in the placeholder when we run the survey. This allows us to administer multiple versions of a question with different inputs all at once. A common use case for this is performing [data labeling tasks](https://docs.expectedparrot.com/en/latest/notebooks/data_labeling_example.html) designed as questions about one or more pieces of textual data that can be inserted into the survey question texts. [Learn more about using scenarios](https://docs.expectedparrot.com/en/latest/scenarios.html).

## Example
For purposes of demonstration we use a PDF copy of the first page of the recent paper [Automated Social Science:
Language Models as Scientist and Subjects](https://arxiv.org/pdf/2404.11794) and conduct a survey consisting of several questions about the contents of it:

<img src="automated_social_science_paper.png" width="300px">

Importing the tools:

In [1]:
# pip install edsl

In [2]:
from edsl.questions import QuestionFreeText, QuestionList
from edsl import ScenarioList, Survey

Here we create a survey of questions that we will administer for each page of the PDF. Note that the `from_pdf()` method requires that the scenario placeholders be `{{ text }}` (for regular scenario objects, you can use any placeholder word that you like):

In [3]:
q_summary = QuestionFreeText(
    question_name = "summary",
    question_text = "Briefly summarize the abstract of this paper: {{ text }}"
)

q_authors = QuestionList(
    question_name = "authors",
    question_text = "List the names of all the authors of the following paper: {{ text }}"
)

q_thanks = QuestionList(
    question_name = "thanks",
    question_text = "List the names of the people thanked in the following paper: {{ text }}"
)

survey = Survey([q_summary, q_authors, q_thanks])

Next we create a `ScenarioList` for the PDF using the `from_pdf()` method, which automatically creates a list of `Scenario` objects for the pages of the PDF which will be inserted in our questions (in our example, this is just the first page of the paper):

In [4]:
automated_social_scientist = ScenarioList.from_pdf("automated_social_scientist.pdf")

Now we can add the list of scenarios to to the survey and run it:

In [5]:
results = survey.by(automated_social_scientist).run()

We can see a list of all the components of results that are directly accessible:

In [6]:
results.columns

['agent.agent_instruction',
 'agent.agent_name',
 'answer.authors',
 'answer.summary',
 'answer.thanks',
 'comment.authors_comment',
 'comment.thanks_comment',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.authors_system_prompt',
 'prompt.authors_user_prompt',
 'prompt.summary_system_prompt',
 'prompt.summary_user_prompt',
 'prompt.thanks_system_prompt',
 'prompt.thanks_user_prompt',
 'question_options.authors_question_options',
 'question_options.summary_question_options',
 'question_options.thanks_question_options',
 'question_text.authors_question_text',
 'question_text.summary_question_text',
 'question_text.thanks_question_text',
 'question_type.authors_question_type',
 'question_type.summary_question_type',
 'question_type.thanks_question_type',
 'raw_model_response.authors_raw_model_response',
 'raw_model_response.summary_

We can select components of the results to inspect and print:

In [7]:
results.select("summary", "authors", "thanks").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ answer                              ┃ answer                              ┃ answer                              ┃
┃ .summary                            ┃ .authors                            ┃ .thanks                             ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ The abstract describes a novel      │ ['Benjamin S. Manning', 'Kehang     │ ['Drew Houston', 'Jordan            │
│ method for generating and testing   │ Zhu', 'John J. Horton']             │ Ellenberg', 'Benjamin Lira          │
│ social science hypotheses using     │                                     │ Luttges', 'David Holtz', 'Bruce     │
│ large language models (LLMs) and    │                                     │ Sacerdote', 'Paul Röttger',         │
│ structural causal models (SCMs).    │                                     │ 'Mohammed Alsobay', 'Ray Duch',     │
│ SCMs help articulate hypotheses,    │                                     │ 'Matt Schwartz', 'David Autor',     │
│ construct LLM-based agents, design  │                                     │ 'Dean Eckles']                      │
│ experiments, and analyze data. The  │                                     │                                     │
│ research applies this method to     │                                     │                                     │
│ various scenarios such as           │                                     │                                     │
│ negotiations, bail hearings, job    │                                     │                                     │
│ interviews, and auctions, proposing │                                     │                                     │
│ and evaluating causal               │                                     │                                     │
│ relationships. The findings         │                                     │                                     │
│ indicate that while LLMs can        │                                     │                                     │
│ predict the direction of causal     │                                     │                                     │
│ effects, they struggle with the     │                                     │                                     │
│ magnitude unless informed by a      │                                     │                                     │
│ fitted SCM. The study suggests that │                                     │                                     │
│ LLMs have latent knowledge that can │                                     │                                     │
│ be better utilized when paired with │                                     │                                     │
│ SCMs. The paper includes thanks for │                                     │                                     │
│ support and feedback from various   │                                     │                                     │
│ individuals and mentions that the   │                                     │                                     │
│ authors' contact information, code, │                                     │                                     │
│ and data will be available online.  │                                     │                                     │
└─────────────────────────────────────┴─────────────────────────────────────┴─────────────────────────────────────┘

## Another example
Let's try another example - the (complete) paper [Owning, Using and Renting:
Some Simple Economics of the "Sharing Economy"](https://john-joseph-horton.com/papers/sharing.pdf).

Here we import it and verify that all the pages have been turned into scenarios:

In [8]:
sharing_economy = ScenarioList.from_pdf("sharing_economy.pdf")

In [9]:
len(sharing_economy)

56

In [10]:
sharing_economy[0:2]

ScenarioList([Scenario({'filename': 'sharing_economy.pdf', 'page': 1, 'text': 'Owning, Using and Renting:\nSome Simple Economics of the “Sharing Economy”∗\nApostolos Filippas†\nJohn J. Horton‡\nRichard J. Zeckhauser§\nMay 10, 2019\nAbstract\nNew Internet-based “sharing economy” markets enable consumer-owners to rent out\ntheir durable goods to non-owners. We model such markets, and explore their equilib-\nria both in the short-run, in which ownership decisions are ﬁxed, and in the long-run,\nin which ownership decisions can be changed. We ﬁnd that “sharing economy” markets\nalways expand consumption and increase surplus, but may increase or decrease owner-\nship. Regardless, ownership is decoupled from individual preferences in the long-run,\nas the rental rates and the purchase prices of goods become equal. If there are costs\nof bringing unused capacity to the market, they are partially passed through, creat-\ning a bias towards ownership. To test our theoretical work empirically, we conduct a\nsurvey of consumers, ﬁnding broad support for our modeling assumptions. The survey\nalso allows us to oﬀer a partial decomposition of the bring-to-market costs, based on\nattributes that make a good more or less amenable to being shared.\n∗Thanks to Andrey Fradkin, Samuel Fraiberger, Joe Golden, Ramesh Johari, Arun Sundararajan, and\nHal Varian for helpful discussions and comments. Author contact information and code are currently or will\nbe available at http://www.john-joseph-horton.com/.\n†Fordham University, Gabelli School of Business\n‡New York University, Stern School of Business\n§Harvard University, Kennedy School of Government\n1\n'}), Scenario({'filename': 'sharing_economy.pdf', 'page': 2, 'text': '1\nIntroduction\nIn traditional rental markets owners hold assets to rent them out. In recent years, ﬁrms\nhave created a new kind of rental market, in which owners sometimes use their assets for\npersonal consumption, and sometimes rent them out. Such markets are commonly referred\nto as peer-to-peer (P2P) rental or “sharing economy” markets. To be sure, some renting\nby consumer-owners has long existed, but given the high transaction cost per rental, it was\nlargely conﬁned to expensive, infrequently used goods, such as vacation homes and pleasure\nboats, usually with rental periods of longer duration. More often, goods were shared among\nfamily and friends, often without explicit payment. In contrast, these new P2P rental markets\nare open markets, and the good is “shared” in exchange for payment.\nAirbnb is a prominent example of a P2P rental market, enabling individuals to rent out\nspare bedrooms, apartments, or entire homes. Airbnb and platforms like it have been her-\nalded by many, as they promise to expand access to goods, diversify individual consumption,\nbolster eﬃciency by increasing asset utilization, and provide income to owners (Botsman and\nRogers, 2010; Edelman and Geradin, 2015; Sundararajan, 2016). The business interest in\nthese platforms has been intense.1\nCompanies organizing “sharing economy” markets have also attracted substantial policy\ninterest, much of it negative (Malhotra and Van Alstyne, 2014; Avital et al., 2015; Slee, 2015;\nFilippas and Horton, 2018). Critics charge that the primary competitive advantage of these\nplatforms is their ability to duck costly regulations—regulations that protect third-parties\nand remedy market failures.2 However, the counter-argument is often made that existing\nregulations were designed to solve market problems that these “sharing economy” platforms\nsolve in an innovative fashion, primarily with better information provision and reputation\nsystems, thereby making top-down regulation unnecessary (Koopman et al., 2014).\nProgress in designing and operating P2P rental markets, as well as in advancing the\ncorresponding policy debate, requires a better understanding of these markets. More speciﬁ-\ncally, what are the economic problems that P2P rental markets address

Let's see what pages are the most important. We start by generating a summary of the paper based on the abstract, using just the first scenario, which is the first page of the paper. We can also create an agent with a relevant persona for the model to use in answering the questions ([learn more about creating AI agents to answer survey questions](https://docs.expectedparrot.com/en/latest/agents.html)):

In [11]:
from edsl.questions import QuestionFreeText, QuestionList, QuestionLinearScale
from edsl import Agent, Survey

In [12]:
social_scientist_agent = Agent({"persona":"You are an experienced social scientist."},
                               instruction = "You are evaluating the contents of a research paper.")

In [13]:
q_summary = QuestionFreeText(
    question_name = "summary",
    question_text = "Draft a summary of the paper based on the abstract: {{ text }}"
)

q_authors = QuestionList(
    question_name = "authors",
    question_text = "List the authors of this paper: {{ text }}"
)

survey = Survey([q_summary, q_authors])

In [14]:
results = survey.by(sharing_economy[0]).by(social_scientist_agent).run()

results.select("summary", "authors").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ answer                                                 ┃ answer                                                 ┃
┃ .summary                                               ┃ .authors                                               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ The paper by Filippas, Horton, and Zeckhauser examines │ ['Apostolos Filippas', 'John J. Horton', 'Richard J.   │
│ the economic implications of the sharing economy,      │ Zeckhauser']                                           │
│ where consumer-owners can rent out durable goods to    │                                                        │
│ non-owners through Internet-based markets. The authors │                                                        │
│ develop a model to analyze the market equilibria in    │                                                        │
│ both short-run (with fixed ownership decisions) and    │                                                        │
│ long-run (with variable ownership decisions)           │                                                        │
│ scenarios. They find that sharing economy markets      │                                                        │
│ invariably increase consumption and overall surplus.   │                                                        │
│ However, the effect on ownership is ambiguous; it may  │                                                        │
│ rise or fall. In the long run, the model predicts that │                                                        │
│ ownership preferences become irrelevant as rental      │                                                        │
│ rates and purchase prices converge. The paper also     │                                                        │
│ acknowledges that costs associated with bringing       │                                                        │
│ unused capacity to the market can lead to a partial    │                                                        │
│ pass-through, which biases the market towards          │                                                        │
│ ownership. The authors' empirical work, which includes │                                                        │
│ a consumer survey, supports the theoretical            │                                                        │
│ assumptions and offers insight into the costs of       │                                                        │
│ bringing goods to the sharing economy, highlighting    │                                                        │
│ characteristics that influence an item's shareability. │                                                        │
└────────────────────────────────────────────────────────┴────────────────────────────────────────────────────────┘

We'll use the summary as context for a new set of questions prompting the agent to identify the most important idea on each page, then select and summarize the most important ideas, and rate the relative importance of each page of the paper:

In [15]:
summary = results.select("summary").first()

In [16]:
q_idea = QuestionFreeText(
    question_name = "idea",
    question_text = "Paper summary: " + summary + 
    " Quote the most important sentence on this page: {{ text }}"
)

In [17]:
ideas = q_idea.by(sharing_economy).by(social_scientist_agent).run().select("idea").to_list()
ideas[0:10]

['We find that “sharing economy” markets always expand consumption and increase surplus, but may increase or decrease ownership. Regardless, ownership is decoupled from individual preferences in the long-run, as the rental rates and the purchase prices of goods become equal.',
 'The goal of this paper is to provide answers to these questions.',
 'Our first major question is why P2P rental markets only became a force in the 21st century, despite the fact that the economic problem these markets are able to solve—under-utilization of durable goods—is hardly new.',
 'While ownership may increase or decrease in the long-run, the option of renting out an owned good makes ownership more valuable. As such, a P2P rental market can have a market-expanding effect, in the sense that it allows a previously infeasible product market to emerge.',
 'owners with lower valuations are the biggest beneﬁciaries, as they consume the good less of the time, and hence they have more excess capacity to rent. Si

In [18]:
q_important = QuestionFreeText(
    question_name = "important",
    question_text = f"""Paper summary: {summary}
    Consider the following ideas that are mentioned in the paper and 
    summarize the 5 most important of them: {ideas}."""
)

In [19]:
results = q_important.by(social_scientist_agent).run()
results.select("important").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ answer                                                                                                          ┃
┃ .important                                                                                                      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ The paper by Filippas, Horton, and Zeckhauser provides a comprehensive analysis of the sharing economy's impact │
│ on consumption, ownership, and market equilibria. The five most important ideas from the paper are: (1) The     │
│ sharing economy invariably increases consumption and surplus, with the ambiguous effect on ownership, which may │
│ rise or fall. In the long run, ownership preferences become irrelevant as rental rates and purchase prices      │
│ converge. (2) Peer-to-peer (P2P) rental markets have become significant due to decreased transaction costs and  │
│ the accumulated experience from e-commerce platforms. These markets utilize under-utilized durable goods by     │
│ allowing owners to rent them out, increasing the value of ownership. (3) The sharing economy benefits both      │
│ owners with lower valuations, who can monetize their excess capacity, and non-owners with higher valuations,    │
│ who gain access to goods they value without the need to own them. (4) The economic model developed in the paper │
│ indicates that bringing unused capacity to the market incurs costs, which leads to partial pass-through in      │
│ rental rates and biases the market towards ownership. (5) Empirical evidence from consumer surveys supports the │
│ theoretical model, showing that planned usage rather than income is the primary determinant of ownership for    │
│ most goods, and that characteristics such as predictability and chunkiness of use influence an item's           │
│ shareability and the likelihood of ownership.                                                                   │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [20]:
important_ideas = results.select("important").first()

In [21]:
q_relative = QuestionLinearScale(
    question_name = "relative",
    question_text = "Consider the following paper summary: " + important_ideas +
    " What is the relative importance of this page of the paper: {{ text }}",
    question_options = [0,1,2,3,4,5],
    option_labels = {0:"Unimportant", 3:"Important", 5:"Most important"}
)

In [22]:
results = q_relative.by(sharing_economy).by(social_scientist_agent).run()

We can filter and sort pages based on the responses, and inspect the agent's comments on its answers:

In [23]:
(results
 .sort_by("page")
 .filter("relative == '5'")
 .select("page", "relative", "relative_comment")
 .print(format="rich")
)

┏━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ scenario ┃ answer    ┃ comment                                                                                  ┃
┃ .page    ┃ .relative ┃ .relative_comment                                                                        ┃
┡━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 5        │ 5         │ This page of the paper appears to be quite important as it provides key insights into    │
│          │           │ how bring-to-market (BTM) costs influence the dynamics of the sharing economy. It        │
│          │           │ discusses the implications of these costs on the rental market's viability, the rental   │
│          │           │ rates, and the decision-making process regarding ownership versus renting. The page      │
│          │           │ connects the theoretical model with practical considerations and outlines the conditions │
│          │           │ under which the sharing economy is most beneficial. The impact of BTM costs on consumer  │
│          │           │ behavior, market equilibrium, and the potential for profit in rental businesses are all  │
│          │           │ critical elements for understanding the sharing economy. Therefore, this page is likely  │
│          │           │ to be of high importance in the context of the paper.                                    │
├──────────┼───────────┼──────────────────────────────────────────────────────────────────────────────────────────┤
│ 14       │ 5         │ This page of the paper appears to be crucial as it outlines the mathematical model that  │
│          │           │ underpins the theoretical framework of the paper. It provides the utility functions for  │
│          │           │ both owners and renters within the sharing economy, which is essential for understanding │
│          │           │ the economic implications of peer-to-peer rental markets. The model also delineates the  │
│          │           │ conditions under which owners and non-owners decide to rent or rent out goods,           │
│          │           │ respectively. Furthermore, the section on short-run equilibrium introduces important     │
│          │           │ concepts for understanding market dynamics in the presence of fixed ownership. The       │
│          │           │ complexity and relevance of the mathematical model to the paper's aims make this page    │
│          │           │ highly important.                                                                        │
├──────────┼───────────┼──────────────────────────────────────────────────────────────────────────────────────────┤
│ 20       │ 5         │ The page provides a detailed analysis of how the P2P rental market affects the utility   │
│          │           │ of both owners and non-owners, showing the differential impact on consumption and        │
│          │           │ utility based on the valuation of the goods. This analysis is crucial for understanding  │
│          │           │ the distributional consequences of the sharing economy and the dynamics of ownership and │
│          │           │ renting. It also touches upon the long-term equilibrium effects and the behavioral       │
│          │           │ adjustments of consumers near the extensive margin, which are significant insights for   │
│          │           │ comprehending the broader economic implications of the sharing economy. Therefore, the   │
│          │           │ content of this page is highly relevant and integral to the paper's overall argument and │
│          │           │ findings.                                                                                │
├──────────┼───────────┼──────────────────────────────────────────────────────────────────────────────────────────┤
│ 23       │ 5         │ This page is crucial as it disc

The selected page:

In [24]:
(results
 .sort_by("page")
 .filter("relative == '5'")
 .select("page", "text")
 .print(format="rich")
)

┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ scenario ┃ scenario                                                                                             ┃
┃ .page    ┃ .text                                                                                                ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 5        │ owners with lower valuations are the biggest beneﬁciaries, as they consume the good less             │
│          │ of the time, and hence they have more excess capacity to rent. Similarly, non-owners with            │
│          │ higher valuations see the largest increase in surplus. As such, the greatest gains in surplus        │
│          │ are obtained when original non-owners value the good nearly as highly as owners, suggesting          │
│          │ that goods where income (rather than taste or planned usage) primarily explains ownership            │
│          │ could oﬀer the greatest increase in surplus.                                                         │
│          │ Although we began by assuming that owners can rent out their unused capacity costlessly,             │
│          │ in practice, making a good available for rentals is at least somewhat costly—as we argued, one       │
│          │ could conceptualize the rise of the “sharing economy” as caused by a signiﬁcant decrease in          │
│          │ these costs. Some of these bring-to-market (BTM) costs are straightforward, such as labor,           │
│          │ depreciation, and complementary consumables. For example, driving with Uber requires                 │
│          │ labor, increases the car’s mileage, and consumes gas.                                                │
│          │ In our model, when we assume that owners do face BTM costs, the predictions change                   │
│          │ in several important ways. We ﬁnd that if BTM costs are suﬃciently high relative to the              │
│          │ purchase price of the good, a P2P rental market cannot be supported at all. If the market            │
│          │ can exist, BTM costs lower the quantity of the good transacted in the market and raise the           │
│          │ rental rate, both in the short- and the long-run. In particular, we show that BTM costs              │
│          │ do get incorporated into rental rates, being the equivalent of a per-unit sales tax. As with         │
│          │ a sales tax, they are not fully incorporated in the rental rate—the magnitude of the pass-           │
│          │ through depends on the elasticity of the supply (owners) and the elasticity of the demand            │
│          │ (renters). An implication of the incomplete pass-through is that both owning and renting             │
│          │ become less compelling as BTM costs increase. Furthermore, total ownership may either                │
│          │ increase or decrease in the long-run as BTM costs change, depending on the incidence of the          │
│          │ BTM costs.                                                                                           │
│          │ When making a good available to be rented is costless, the rentals option decouples in-              │
│          │ dividual preferences from ownership. However, when BTM costs are introduced, the incom-              │
│          │ plete pass-through of BTM costs couples preferences and ownership again, tilting consumers           │
│          │ with higher valuations towards ownership. The reason consumers with higher valuations—               │
│          │ and hence more planned usage—ﬁnd ownership relatively more attractive than owners with               │
│          │ low valuations, is that consumers bear no BTM costs for own-usage. This is similar to the            │
│          │ ineﬃcient bias towards home production that

Please see our [documentation page](https://docs.expectedparrot.com/) for examples of other survey methods and use cases!